# 클러스터링(clustering)

클러스터링이란 유사한 데이터를 서로 그룹지어주는 것으로 회귀, 분류 와 함께 머신러닝에서 가장 중요한 문제중에 하나이다. 인간은 유사한 경험, 대상을 서로 연관짓고 묶어주며 학습이 이루어지는 경우가 많다. 그렇기 때문에 지능적인 프로세스에 있어서 이러한 클러스터링은 수없이 많은 분야에 적용되고 응용된다. 

어찌보면 간단할 수도 있다고 생각하기 쉽지만 실제로 '비슷한 데이터' 라는 기준이 매우 모호하며 그룹화시키는 데이터나 목적 또한 다양하기 때문에 이에 맞는 여러가지 클러스터링 알고리즘이 사용되며 지금도 활발히 연구되는 분야중에 하나이다. 

<hr style="height:3px">

# kmeans 

가장 오래된 클러스터링 방법중에 하나로서 그 심플하면서도 빠른 속도로 지금도 사랑받고 있는 알고리즘이다. kmeans 는 다음과 같은 방법으로 수행된다. 

1. 랜덤하게 k개의 중점을 선정한다. 
2. 모든 점들은 자신과 가장 가까운 중점에 소속된다.
3. 각각의 중점을 자신에게 소속된 점들의 표준값(무게중심) 으로 이동한다. 
4. 중점의 변화가 없을때까지 2~3을 반복한다. 

아래 그림을 보면 좀더 이해가 빠를 것이다. 


<img src = "https://cdn-images-1.medium.com/max/1200/0*rrzG3LyOnAvOepbJ.png" 
    style = "float:left"     
    title = "출처: https://cdn-images-1.medium.com/max/1200/0*rrzG3LyOnAvOepbJ.png" > 


그림에서 k=2 .. 즉 처음 두 중점을 기준으로 두 그룹이 생성되고(step1) 생성된 그룹에 의해 다시 중점이 이동한다. (step2 ) , 이제 다시 바뀐 중점에 의해 그룹을 재조정하고 (step3) 이를 반복한다.  마지막에는 (step5) 그룹의 변화가 없기때문에 중점도 그대로인 상태가 되어 알고리즘이 종료되고 최종 그룹이 결정된다. 

사실 sklean 에서는 당연히 이런 kmeans 함수를 제공한다. 하지만 kmeans 는 클러스터링의 가장 기본이 되는 기법이며 코드도 매우 간결하기 때문에 kmeans 작동과정을 확인하기 위해서도 이를 직접 작성해보도록 하자. 


<hr>

## kmeans 직접 만들어보기 

### 데이터 군 만들기

먼저 4가지 데이터 군읊 만듪어보도록 하자. numpy 시간에 군집을 np.random.normal 배운 기억이 있을 것이다. 이번에는 좀더 간단하게 여러 군집을 만들 수 있는 make_bolos 을 사용해보도록 하자. (sklearn.datasets.samples_generator)

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
 make_blobs(n_sample = 총 점의 숫자, centers=군집의 숫자, cluster_std=군집분산, 
</div>

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets.samples_generator import make_blobs

x,y = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.8, random_state=16)
print(x, y)


make_blobs 에 의해 생성된 군집은 x = [[x1, y1], [x2, y2] , ... ] 이런 형태를 가진다. 그리고 y에는 각 소속 군집의 인덱스 (0~3) 이 들어있다. 
간단히 군집을 표시하려면 plot 으로 표기할 수 있으나 이번에는 각 군집의 인덱스별로 컬러를 다르게 표기하기 위해 plot 대신 scatter 를 사용하도록 하겠다 

- plt.scatter( X축 데이터, Y축 데이터, c = label 데이터 ) 


In [ ]:
plt.scatter( x[:,0], x[:,1], c = y )

### 랜덤하게 초기 중점 구하기 

이번에는 랜덤하게 4개의 점을 골라서 이를 중점으로 선정하도록 한다. 

In [ ]:
import numpy as np
np.random.shuffle(x)
center = x[0:4,:]
print(center)

이번에는 이것을 -8 ~ 8 로 변환하여 화면에 표시하자 

In [ ]:
#center = center * 16.0
plt.scatter( x[:,0], x[:,1] )
plt.plot(center[:,0], center[:,1], '+', markersize = 10, color = 'red')

### 중점별로 군집 나누기 

이제 군집별로 가장 가까운 center 를 골라 이에 소속되도록 한다. for문을 열심히 돌리면 되겠지만.. 친절한 파이썬은 이를 위한 함수가 준비되어있다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
 pairwise_distances_argmin(A, B) : A의 값들마다 가장 가까운 B의 값을 찾아 그 인덱스의 배열을 리턴한다. 
</div>

요컨데 다음과 같은 문장으로 우리는 각 점들이 4개의 센터중 어디에 가장 가까운지 그 배열을 한번에 얻을 수 있다. 

In [ ]:
from sklearn.metrics import pairwise_distances_argmin
label = pairwise_distances_argmin(x, center)
print(label)

이것을 다시 화면에 그림으로 표시하면 각 center 마다 소속 그룹을 색별로 표시  가능하다. 


In [ ]:
print(center)
plt.scatter(x[:,0], x[:,1], c = label)
plt.scatter(center[:,0], center[:,1], marker = '+', s = 100, c = 'red')


### 새로운 중점 구하기 


이제 center마다 소속 그룹들의 중점을 새로 구한다. 기존의 프로그래밍방식이라면  다음과 같은 코드가 필요하다. 

In [ ]:
new_center = center.copy()

for gi in range(4):  # 그룹별로
    group = []
    for xi in range( len(x) ) : # 점들마다
        if label[xi] == gi :    # 해당 그룹 소속일경우 그룹에 추가                    
            group.append(x[xi,:])
            
    #  새로운 센터를 계산
    if len(group) > 0 :         
        new_center[gi] = np.array(group).mean() 
    

np array 의 다음의 기능을 사용하면 이를 좀더 간략화 시킬 수 있다. 

In [ ]:
a  = np.array([1,2,3])
print( a[[True,False,True]] )

i = np.array( [0,1,0] )
print( i == 0 )
print( a[i==0] )

이제 이 기능을 사용하여 더 간단한 위의 코드를 더 간단히 바꿔보자. 

In [ ]:
new_center = center.copy()
for gi in range(4) :
    new_center[gi] = np.array( x[label == gi] ).mean(0)

이제 바뀐 센터를 화면에 찍어보자. 

In [ ]:
plt.scatter(x[:,0], x[:,1], c = label)
plt.scatter(new_center[:,0], new_center[:,1], marker = '+', s = 100, c = 'red')

### 그룹 재생성 -> 중점 구하기 반복

센터가 각 소속그룹의 중앙으로 이동한 것을 볼 수 있다. 이제 이것을 중점의 이동이 없을때까지 반복해보자. 중점의 변화가 없음은 다음과 같은 코드로 체크할 수 있다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
if np.all( new_center == center ) :
</div>

np.all 은 그 모든 인자가 True 일때 True 를 리턴하므로 center 와 new_center 가 동일한 경우에만 작동한다. 이를 종료조건으로 두면 다음과 같다. 

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

while True:    
    label = pairwise_distances_argmin(x, center)
    new_center = center.copy()
    for gi in range(4) :
        new_center[gi] = np.array( x[label == gi] ).mean(0)
        
    if np.all(new_center == center) :
        break
    else :
        center = new_center
    
    plt.scatter(x[:,0], x[:,1], c = label)
    plt.scatter(new_center[:,0], new_center[:,1], marker = '+', s = 100, c = 'red')
    plt.show()


잘 작동된다면 kmeans 가 작동하는 과정을 그래픽으로 확인 가능할 것이다. 


### <font color = 'red'> 연습 4-1: kmenas 함수화 </font>

위의 kmeans 를 다음과 같이 함수화시킨 후에 다음 데이터를 클러스터링 해보자 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    labels, centers = kmeans( x, k ) : x =  입력 데이터, k = 중점의 숫자
</div>

리턴 : 
- labels : 각 x memeber 의 그룹 인덱스
- centers : 그룹별 중심

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets.samples_generator import make_blobs

x,y = make_blobs(n_samples=400, centers=6,
                       cluster_std=0.5, random_state=11)

plt.scatter(x[:,0], x[:,1], s=8)
plt.show()


### <font color = 'red'> 연습 4-2: 3차원 데이터 클러스터링  </font>

이번에는 3차원 데이터를 클러스터링 할 수 있도록 kmeans3d 함수를 만들어서  다음 데이터를 클러스터링해보자 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    kmeans3d( x, k ) : x =  입력 데이터(3d), k = 중점의 숫자
</div>



In [ ]:
from mpl_toolkits.mplot3d import Axes3D

ax = plt.figure().gca( projection='3d')
x,y = make_blobs(n_samples=300, centers=5,n_features = 3,
                       cluster_std=0.8, random_state=15)
ax.scatter(x[:,0], x[:,1], x[:,2], c = y)

In [ ]:
#연습 4.2의 코드를 여기에 작성해보세요

<hr>

## sklearn 의 kmeans 라이브러리 사용법

sklearn 은 다양한 클러스터링 기법을 제공하며 kmeans 도 당연히 기본적으로 제공한다. 이제 이것을 이용하여 kmeans 클러스터링을 수행하는 예를 보도록 하자. 


In [ ]:
from sklearn import cluster
km = cluster.KMeans( n_clusters = 4 )

x,y = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.8, random_state=16)

km.fit(x)
plt.scatter(x[:,0], x[:,1], marker = '.', c = km.labels_)




linear_regression 을 이미 접하였기에 이해하기 상당히 쉬울 것이다. 

다음 구문은 km 이라는 이름으로 kmeans 모델을 생성한다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    km = cluster.Kmeans( n_clusters = 클러스터 숫자 )
</div>

모델이 생성되면 다음의 구문으로 간단히 클러스터링을 수행할 수 있다. 데이터 배열이  몇차원인가는 자동으로 분석하므로 따로 입력할 필요는 없다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    km.fit( 데이터 배열 ) 
</div>

클러스터링의 결과는 간단하게 각 데이터에 대한 label 값으로 km.labels_ 에서 배열로 얻을 수 있다. 위에서 이미 자주 사용했듯이 scatter 함수의 색상 파라미터 c 값에  labels_ 를 입력해서 손쉽게 다음과 같이 출력할 수 있다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    plt.scatter(x[:,0], x[:,1], marker = '.', c = km.labels_)
</div>



### <font color = 'red'> 연습 4-3: sklearn 을 이용한 3차원 데이터 클러스터링  </font>

익숙해지는 의미에서 4-2의 3차원 데이터를 sklearn.clustewr.Kmeans 모델을 이용해서 클러스터링하고 이를 시각화시켜보자. 


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

ax = plt.figure().gca( projection='3d')
x,y = make_blobs(n_samples=300, centers=5,n_features = 3,
                       cluster_std=0.8, random_state=15)
ax.scatter(x[:,0], x[:,1], x[:,2], c = y)

In [ ]:
# 연습 4-3 의 코드를 여기에 작성하세요

<br><br><br>





<hr style="height:3px">

# Meanshift clustering

단어 그대로 중심이동에 의한 클러스링이다. 각 점들이 산을 올라가듯이 점점 밀집도가 높은 곳으로 이동하여 뭉치는 것을 기반으로 한다. 

<img src = "https://frnsys.com/ai_notes/assets/meanshift_bw_02.png" 
    style = "float:left"     
    title = "출처: https://frnsys.com/ai_notes/assets/meanshift_bw_02.png" > 
<center> meanshift 클러스터링 과정 </center>



 <br>
 <br>

## Meanshift

meanshift 는 군집의 중앙을 찾아가는 가장 단순한 방법중 하나이다. 각 점이 다음과 같은 과정을 반복하면서 이뤄진다. 

1. 주변 범위의 점들의 집합을 찾는다. 
2. 찾은 점들의 평균(중심점)으로 이동한다. 
3. 중심의 이동이 없을때까지 1-2를 반복한다. 


<img src = "http://tw.rpi.edu/media/latest/mean_shift_illustration_small.png" 
    style = "float:left"     
    title = "출처: http://tw.rpi.edu/media/latest/mean_shift_illustration_small.png" >        


<br>

먼저 Meanshift 의 과정을 직접 작성해보자.


먼저 x[0] 이 중심을 찾아가는 과정부터 생성해보도록 하겠다. 먼저 데이터를 만들고 시작점을 그려보도록  한다. 


In [ ]:
x,_ = make_blobs(n_samples=100, centers=1, n_features = 2,
                       cluster_std=0.1, random_state=15)
px = x[0]
d = 0.1
plt.scatter(x[:,0], x[:,1], marker = '.',)
plt.scatter(px[0],  px[1], marker = '+')
           
circle = plt.Circle((px[0], px[1]), d, color = 'r', fill = False)
plt.gca().add_artist(circle)

plt.show()


이제 px 의 주변 점들을 구해보자. 


In [ ]:
nx =  x[np.sqrt( np.sum( ( x - px ) ** 2 , axis = 1)) < d]

이제 점을 이동하고 이동된 새로운 점을 출력한다. 

In [ ]:
npx = np.mean(nx, axis = 0)

# 시각화
plt.plot(x[:,0], x[:,1], '.',)
plt.plot(nx[:,0], nx[:,1], '+')
plt.plot(px[0],  px[1], 'o')
plt.plot(npx[0],  npx[1], 'o')

px = npx

#새로운 중점 시각화
circle = plt.Circle((px[0], px[1]), d, color = 'r', fill = False)
plt.gca().add_artist(circle)

이제 이를 반복하는 코드를 작성해보자. 안정상태, 즉 더이상 중심이 변하지 안는 상태에 도달하게 되면 반복을 탈출하게 하면  된다. 

In [ ]:
while True: 
    nx =  x[np.sqrt( np.sum( ( x - px ) ** 2 , axis = 1)) < d ] 
    npx = np.mean(nx, axis = 0)
    if np.all(px == npx) : 
        break
    px = npx    
    
plt.plot(x[:,0], x[:,1], '.',)
plt.plot(px[0],  px[1], 'o')
   

모다시피 점들의 중앙을 찾은 것을 볼 수 있다. 


<hr>

## Meanshift 로 클러스터링 하기

Meanshift clustering 은 위의 meanshift 를 모든 점에 대해서 수행하여 같은 점으로 모이는 점들을 하나의 군집(클러스터)로 보는 것이다. 
이제 클러스터링 알고리즘을 작성해 보자.  먼저 이를 위해서 간단한 두 군집으로 나누도록 하겠다. 



In [ ]:
x,_ = make_blobs(n_samples=200, centers=2, n_features = 2,
                       cluster_std=1, random_state=2)
plt.scatter(x[:,0], x[:,1], marker = '.',)

이제 모든 점들에 대해 meanshift 를 수행하고 이를 mx라는 배열에 저장한다. 

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

labels = np.zeros( len(x) ) # 클러스터 인덱스 저장장소
mx = []
d = 2
for px in x :
    while True: 
        nx =  x[np.sqrt( np.sum( ( x - px ) ** 2 , axis = 1)) < d ] 
        npx = np.mean(nx, axis = 0)
        if np.all(px == npx) : 
            break
        px = npx  
    mx.append(px)
    
mx = np.array(mx)
plt.plot(x[:,0], x[:,1], '.')
plt.plot(mx[:,0], mx[:,1], 'o')


mx 의 경우 몇몇 점으로 수렴해서 모인 것을 볼 수 있다. 

이제 클러스터번호 label 을 생성하자. 이때 unique 와  kmeans 에서 사용되었던 pairwise_distance_argmin 을 사용하면 보다 더 간단히 이를 수행할 수 있다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
 np.unique(A, axis = 축) : array A의 값들중에 중복을 제거한다. axis 값이 있을 경우 그 축을 기준으로 중복을 제거한다. 
</div>

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
 pairwise_distances_argmin(A, B) : A의 값들마다 가장 가까운 B의 값을 찾아 그 인덱스의 배열을 리턴한다. 
</div>


In [ ]:
centers = np.unique(mx, axis = 0) 
labels = pairwise_distances_argmin(mx, centers)
plt.scatter(x[:,0], x[:,1], c = labels)
plt.show()        

<hr>

## 가우시안 커널

meanshift 는 kmeans 보다 더 정확한 군집을 찾아내지만 그 성능이 탐색범위 (bandwidth) 에 의해 지나치게 좌우되는 단점이 있다. 이러한 단점을 보완하기 위해 새로운 중점을 구할때 현재의 중점에 가까울수록 그 가중치를 높이는 방법을 사용하는데 특히 정규분포를 이용해 점마다 가중치를 주는 기법을 가우시안 커널(gaussian kernel) 이라고 한다. 기본 meanshift 가 모든 범위 내의 점들에 대해 동일한 가중치로 평균을 구한다면 가우시안 커널은 중심에 가까운 점들에게 더 높은 가중치를 부여하여 평균을 구한다. 이는 범위값에 의해 지나치게 결과가 좌우되는 것을 막아준다. 


<img src = "https://t1.daumcdn.net/cfile/tistory/2122D950539942B63A" 
    style = "float:left"     
    title = "출처: https://t1.daumcdn.net/cfile/tistory/2122D950539942B63A" > 




간단한 가우시안 1차원 함수의 예를 보자. x 데이터는 -5 ~ 5 까지 100 개로 구성하겠다.  이때 중심을 0으로 하고 표준편차를 2로 가우시안 커널을 각 점마다 대입시키면 다음과 같이 된다. 

In [ ]:
x = np.linspace(-5,5, 100)
m,d = 0, 2
gw = np.exp(-0.5 * (( x - m ) / d) ** 2 ) / ( d * np.sqrt( 2 * np.pi ))
plt.plot(x, gw, '.')


이제 다음과 같은 2차원 점들에 가우시안을 적용시켜보자. -5 부터 5까지 0.5 단위로 그리드를 생성하였다. 

In [ ]:
n = 10
x = np.mgrid[-5:5.1:0.5, -5:5.1:0.5].reshape(2,-1).T # 그리드형태의 데이터 생성 
plt.plot( x[:,0], x[:,1], '.')

위의 데이터의 중점은 0,0 에 근접하게 된다. 이번에는 이를 중점(m)으로 하고 표준편차는 마찬가지 2로 잡고 가우시안 함수를 적용해보도록 하겠다. 

이 경우 (x - m) 는 x,y 에서  중점(mx,my) 까지의 거리 $ \sqrt{ (x - mx)^2 + (y - my)^2} $ 가 된다. 지나치게 복잡해지는 것을 막기 위해 이를 dx 로 생성하겠다. 그리고  (x - m) 대신 dx 를 넣어주면 다음과 같다. 

In [ ]:
m = np.array([0,0])
d = 2
dx = np.sqrt( np.sum( ( x - m ) ** 2 , axis = 1 ) )
gw = np.exp(-0.5 * (dx / d) ** 2 ) / ( d * np.sqrt( 2 * np.pi ))

이제 이것을 시각화 해주면 다음과 같다. 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
ax = plt.figure().gca( projection='3d')
plt.plot(x[:,0], x[:,1], gw, '.')

## meanshift 에 가우시안 커널 적용

가우시안 함수를 사용하지 안는 기본 meanshift 경우는 다음과 같은 코드로  중심이동이 이루어졌다. 먼저 반경 d 안의 점들(nx)을 찾는다
 
<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
nx =  x[np.sqrt( np.sum( ( x - px ) ** 2 , axis = 1)) < d ]                                                 

</div>

이를 가우시안 커널을 사용하여 중점은 px 로 표준편차는 탐색반경의 1/2 로 설정하면 다음과 같이 변경된다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px"> 
kd = d / 2 <br>
nx =  x[np.sqrt( np.sum( ( x - px ) ** 2 , axis = 1)) < d ] <br>                                                       
dx = np.sqrt( np.sum( ( nx - px ) ** 2 , axis = 1 ) ) # 현재 중심 px와 주변점들(nx)의 거리 <br>
gw = np.exp(-0.5 * (dx / kd) ** 2 ) / ( kd * np.sqrt( 2 * np.pi )) # 거리를 기준으로 가우시안 가중치생성 <br>
gw = gw.reshape(len(gw),1) <br>
</div>

이제 이를 이용해  가우시안 커널을 적용한 새로운 중점을 구하기 위해 가우시안 가중치를 곱한 후에 가중치의 합을 나눈다. 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px"> 
npx = np.sum(nx*gw, axis = 0) / np.sum(gw) 
</div>

이를 적용한 코드는 다음과 같다. 

In [ ]:
# from sklearn.metrics import pairwise_distances_argmin
x,_ = make_blobs(n_samples=200, centers=2, n_features = 2,
                       cluster_std=1, random_state=2)

labels = np.zeros( len(x) ) # 클러스터 인덱스 저장장소
mx = []

d = 2 # 탐색 반경
kd = d / 2 # 가우시안 커널 표준편차 

for px in x:    
        
    while True:         
        nx =  x[np.sqrt( np.sum( ( x - px ) ** 2 , axis = 1)) < d ] 
        dx = np.sqrt( np.sum( ( nx - px ) ** 2 , axis = 1 ) )        
        gw = np.exp(-0.5 * (dx / kd) ** 2 ) / ( kd * np.sqrt( 2 * np.pi ))
        gw = gw.reshape(len(gw),1) 
        npx = np.sum(nx*gw, axis = 0) / np.sum(gw)       
                
        if  np.sqrt(np.sum((px - npx)**2 )) < 0.001 : 
            break
        px = npx  
    mx.append(npx)
    
# 클래스정보 생성     
mx = np.round( np.array(mx), decimals = 1)
centers = np.unique(mx, axis = 0) 
labels = pairwise_distances_argmin(mx, centers)

# 화면 출력
plt.plot(mx[:,0], mx[:,1], '+')
plt.scatter(x[:,0], x[:,1], marker = '.', c = labels)
plt.show()    


### <font color = 'red'> 연습 4-4: meanshift 함수화 </font>

위의 meanshift 를 다음과 같이 함수화시킨 후에 다음 데이터를 클러스터링하고 시각화해보자

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    labels, centers = meanshift( x, d ) : x =  입력 데이터, d = 탐색범위
</div>

리턴 : 
- labels : 각 x memeber 의 그룹 인덱스
- centers : 그룹별 중심


In [ ]:
# 연습  4-4 
def meanshift(x, d):
    labels, centers = [],[]     
    # 여기에 함수 코드를  작성해보세요
    
    
    return labels, center        
    

In [ ]:
n = 100
x1 = np.random.normal(np.pi / 2, np.pi / 6 ,n)
y1 = np.sin(x1) + np.random.normal(0, np.pi / 100,n)
x = np.append( np.c_[x1,y1], np.c_[y1 + 2, x1 - 1], axis = 0)

plt.scatter(x[:,0], x[:,1], marker = '.',)
plt.show()

In [ ]:
# 연습 4-4 여기에 meanshift 함수를 사용해서 클러스터링을 수행하고 결과를 시각화하세요

<hr>

## sklearn 의 meanshift 라이브러리 사용법

이번에는 sklearn 의 meanshift 사용법을 알아보자. 사실 sklearn 의 clustering 등 거의 유사한 형태이기 때문에 kmeans 대신 meanshift 를 사용하면 된다. 


<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    ms = cluster.MeanShift(bandwidth = 탐색범위)
</div>

단 kmeans 는 k 값이 거의 필수적이지만 meanshift 의 탐색범위 (bandwidth) 는 따로 입력을 하지 안으면 자동으로 그 값을 설정해준다. 이제 kmeans 와 마찬가지로 

<div  style  = "background-color:#EEEEEE; margin:20px; padding:10px">
    ms.fit(x)
</div>
로 클러스터링을 수행하고 ms.labels 로 그룹 인덱스를 사용하면 된다. 

In [ ]:
from sklearn import cluster
ms = cluster.MeanShift()
ms.fit(x)

In [ ]:
plt.scatter(x[:,0], x[:,1], marker = '.', c = ms.labels_)



### <font color = 'red'> 연습 4-5: meanshift 와 kmeans 비교 </font>

다음 url 에 있는 데이터를 read_csv 를 이용해 읽어들이고 kmeans 와 meanshift 양쪽을 사용해 두 클러스터링의 결과를 비교해보자 

http://cs.joensuu.fi/sipu/datasets/Aggregation.txt


<br><br><br>


<hr style="height:3px">

# 이미지 색상 클러스터링 

이번에는 클러스터링으로 조금더 실용적인 예를 다뤄보도록 하자. 다음은 이미지를 읽어들여서 배열로 변환하는 코드이다. 이를 위해 Opencv 라이브러리인 cv2 를 불러들였다. 

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

img = cv2.imread("images/elsa.jpg")
print(img.shape)

img = img[:,:,::-1] # bgr을 rgb로 변환
plt.imshow(img)

print( img.shape )를 통해 294 x 170 x 3 으로 구성된 3차원 배열임을 알 수 있다. opencv 는 기본적으로 numpy 배열을 사용하기 때문에 이제 이 이미지를 다루는 것은 익숙할 것이다.  그러나 이것들을 각각의 색상으로 처리하기 위해서는  ( ? x 3 ) 으로 펴줄 필요가 있다. 


In [ ]:
arr = img.reshape( img.shape[0] * img.shape[1], 3 )
print(arr.shape)
print(arr)


이제 이것은 x,y,z 대신 r,g,b 를 가졌을 뿐 3차원 점들과 같이 취급할 수 있다. 이제 이 점들을 kmeans 에 넣어보겠다. 

In [ ]:
from sklearn import cluster
km = cluster.KMeans(n_clusters = 5)
km.fit(arr)


아마 성공적으로 5개의 클러스터가 생성된 것을 볼 수 있다. 이제 클러스터의 중점은 이 그림을 5가지 대표색이 된다. 하지만 실수로는 사용할 수 없으니 다음과 같이 이를 정수로 만들어준다. 

In [ ]:
print(km.cluster_centers_)
clr = km.cluster_centers_.astype(int)
print(clr)

이를 도식화 시키면 다음과 같다. 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
ax = Axes3D(plt.figure())
ax.scatter(arr[:,0], arr[:,1], arr[:,2], c = km.labels_)
plt.show()

각각의 점들에 대한 중점의 매핑은 km.labels_ 에 저장된 것을 기억할 것이다. 이를 이용해 새로운 이미지를 생성하도록 하겠다. 

In [ ]:
nimg = np.full(arr.shape,[0,0,0])
for i in range(len(km.labels_)):    
    nimg[i] = clr[ km.labels_[i]]

nimg = nimg.reshape(img.shape)
plt.imshow(nimg)


### <font color = 'red'> 연습 4-6: 인터넷에서 이미지를 읽어서 직접 해보기 </font>

다음의 코드는 직접 인터넷에서 url 을 통해 이미지를 로딩하는 코드이다. 아래의 코드를 참조해서 인터넷에 작은 이미지 (최대 가로 x 세로 = 400 x 400 ) 이하의 이지를 찾아 직접 kmeans 를 이용해 이미지를 단순화해보자. 또한 meanshift 도 사용해서 단순화해보자. 


In [ ]:
from urllib import request

req = request.urlopen('https://is3-ssl.mzstatic.com/image/thumb/Purple125/v4/12/17/bb/1217bb95-5d58-1dc4-ada2-bd07f60e3514/AppIcon-1x_U007emarketing-85-220-4.png/246x0w.jpg')
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
img = cv2.imdecode(arr, -1)
img = img[:,:,::-1] # bgr을 rgb로 변환
plt.imshow(img)

In [ ]:
# 연습 4-6 의 코드를 여기에 작성하세요 